<a href="https://colab.research.google.com/github/2001harsh-mehra/vehicle-image-classification/blob/main/1_CNN_without_data_augmentation_Vehicle_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile as vehiclezip
import os as os_veh
import matplotlib.pyplot as pltveh
import matplotlib.image as mpimgveh
import shutil as shut_veh
from sklearn.model_selection import train_test_split as tts_veh
from tensorflow.keras.preprocessing.image import ImageDataGenerator as imgg_gen_veh
import numpy as npveh
from tensorflow.keras.preprocessing.image import load_img as ld_img_veh, img_to_array as img_2_ary_veh

In [ ]:
vehicle_base_drive = '/content/drive/MyDrive/Vehicle Classification'
vehicle_zip_dir = os_veh.path.join(vehicle_base_drive, 'vehicles.zip')
vehicle_extract_dir = '/content/vehicle_data'

with vehiclezip.ZipFile(vehicle_zip_dir, 'r') as zip_ref:
    zip_ref.extractall(vehicle_extract_dir)
print("Unzipping done. Vehicle images are now ready!")

vehicle_main_dir = os_veh.path.join(vehicle_extract_dir, 'Vehicles')

for clas_vehicle in os_veh.listdir(vehicle_main_dir):
    clas_path_veh = os_veh.path.join(vehicle_main_dir, clas_vehicle)
    if os_veh.path.isdir(clas_path_veh):
        pictures = os_veh.listdir(clas_path_veh)
        full_veh_cnt = len(pictures)
        print(f"Vehicle Class: {clas_vehicle} — Total number of images: {full_veh_cnt}")

Data Splitting


In [ ]:
split_output_dir = os_veh.path.join('/content', 'vehicle_data_split')
tt_rto = 0.2  # 20% data taken for testing

# Create train and test folders
for veh_foldr in ['train', 'test']:
    os_veh.makedirs(os_veh.path.join(split_output_dir, veh_foldr), exist_ok=True)

#Splitting and copying vehicle images
for clas_vehicle in os_veh.listdir(vehicle_main_dir):
    clas_path_veh = os_veh.path.join(vehicle_main_dir, clas_vehicle)
    if not os_veh.path.isdir(clas_path_veh):
        continue

    all_pics = os_veh.listdir(clas_path_veh)
    tn_pics, tt_pics = tts_veh(all_pics, test_size=tt_rto, random_state=42)

    for split_type, image_list in [('train', tn_pics), ('test', tt_pics)]:
        dest_direc = os_veh.path.join(split_output_dir, split_type, clas_vehicle)
        os_veh.makedirs(dest_direc, exist_ok=True)
        for picc in image_list:
            shut_veh.copy(os_veh.path.join(clas_path_veh, picc), os_veh.path.join(dest_direc, picc))

            print("Dataset successfully splitted into training and testing sets!")
for split_type in ['train', 'test']:
    print(f"\n {split_type.upper()} SET:")
    split_path = os_veh.path.join(split_output_dir, split_type)
    for clas_vehicle in os_veh.listdir(split_path):
        img_count = len(os_veh.listdir(os_veh.path.join(split_path, clas_vehicle)))
        print(f"{clas_vehicle}: {img_count} images")
    print('-' * 80)

Image Preprocessing

In [ ]:
random_seed = 42

# rescaling the vehicle images
veh_img_genn = imgg_gen_veh(rescale=1./255)
sze_veh = (224, 224) # target size
sze_bch = 32  #batch size
one_hot_veh = 'categorical' # class mode